## Interpolat between multple Frames to create a smooth animation time lapse

In [3]:
import os
from pathlib import Path
import numpy as np
import tempfile
import tensorflow as tf
import mediapy
import cv2
from PIL import Image, ExifTags

from natsort import natsorted
import shutil
from tqdm import tqdm
from datetime import datetime
import subprocess
import math
import mediapy as media
import sys
from typing import Generator, Iterable, List, Optional


from eval import interpolator, util


2025-01-09 12:03:10.273190: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-09 12:03:10.928952: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Config

In [ ]:
# Download the pretrained model
import gdown
os.makedirs('pretrained_models/film_net/Style/saved_model', exist_ok=True)

if os.path.exists('pretrained_models/film_net/Style/saved_model/saved_model.pb'):
    print('Model already downloaded')
else:
  folder_url = 'https://drive.google.com/drive/folders/1i9Go1YI2qiFWeT5QtywNFmYAA74bhXWj'
  gdown.download_folder(folder_url, output='pretrained_models/film_net/Style/saved_model', quiet=False)

In [4]:

MAX_DIM = 1080
MAX_INTERPOLATED_FRAMES = 31  # 1 less than powers of 2
MODEL_PATH = "pretrained_models/film_net/Style/saved_model"

In [5]:
if MAX_INTERPOLATED_FRAMES + 1 not in (2, 4, 8, 16, 32, 64, 128):
    raise ValueError("MAX_INTERPOLATED_FRAMES + 1 must be a power of 2")

In [6]:
input_dir = 'input_frames/falake_window'
output_dir = 'output_frames/lake_window'

processed_input_dir = tempfile.mkdtemp()
temp_interpolated_frames_dir = tempfile.mkdtemp()

frame_interval = 3600 * 24 # gap between frames in seconds

## Plan

In [7]:
# 1. Preprocess images
# 2. for succesive pairs of images:
# 3.     generate interpolated frames
# 4.     add interpolated frames to the output directory
# 5. generate video from output directory

## Utility Functions

In [8]:

# create empty output directory
def create_empty_dir(dir_name):
  if os.path.exists(dir_name):
    shutil.rmtree(dir_name)
  os.makedirs(dir_name)


def resize_and_save(input_path, output_path, size):
  if input_path.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
      img = Image.open(input_path)
      img = img.resize(size, Image.Resampling.LANCZOS)
      img.save(output_path)
  else:
      raise ValueError(f'Unsupported file format: {input_path}')


# prepare input data
def prepare_images(input_dir, processed_input_dir, size):
  files = natsorted([f for f in os.listdir(input_dir) if f.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))])
  if len(files) < 0:
    raise FileNotFoundError('no images found in input directory')
  
  for f in tqdm(files, desc='Preparing images'):
    resize_and_save(os.path.join(input_dir, f), os.path.join(processed_input_dir, f), size)


def get_new_size(image_path: str, max_dim: int = MAX_DIM) -> np.ndarray:
    """Resize the image so that the maximum dimension is `max_dim`."""
    image = cv2.imread(image_path)
    h, w = image.shape[:2]
    if h > w:
        new_h = max_dim
        new_w = int(w * new_h / h)
    else:
        new_w = max_dim
        new_h = int(h * new_w / w)
    return (new_w, new_h)


def string_to_timestamp(datetime_str):
    # Convert the datetime string to a datetime object
    timestamp = datetime.strptime(datetime_str, "%Y%m%d_%H%M%S")
    return timestamp


def timestamp_to_string(timestamp):
    # Convert the timestamp number to a datetime object
    dt = datetime.fromtimestamp(timestamp)
    # Format the datetime object to the desired string format
    formatted_string = dt.strftime("%Y%m%d_%H%M%S")
    return formatted_string


def choose_evenly_spaced_timestamps(timestamps, k):
    # Ensure we have at least k timestamps
    if k > len(timestamps):
        raise ValueError("k cannot be greater than the number of timestamps")

    # Calculate the interval between timestamps
    n = len(timestamps)
    interval = (n - 1) / (k - 1)

    # Select the timestamps
    chosen_timestamps = []
    for i in range(k):
        index = round(i * interval)
        chosen_timestamps.append(timestamps[index])

    return chosen_timestamps


def save_video(frames, out_path):
    ffmpeg_path = util.get_ffmpeg_path()
    mediapy.set_ffmpeg(ffmpeg_path)
    mediapy.write_video(out_path, frames, fps=30)


def save_frames(frames, output_dir, format='jpg'):
    """
    Save interpolated frames to the specified output directory and return the output paths.
    Args:
        frames: List of image arrays
        output_dir: Directory to save frames
        format: Image format to save (jpg/png)
    Returns:
        list: List of file paths where the frames are saved.
    """
    output_paths = []
    for idx, frame in enumerate(frames):
        output_path = os.path.join(output_dir, f'frame_{idx:06d}.{format}')
        util.write_image(output_path, frame)
        output_paths.append(output_path)
    return output_paths


def write_video_from_images(image_dir, output_path, fps):
    image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])
    
    if not image_files:
        print("No images found in the directory.")
        return
    
    # Read the first image to get the dimensions
    first_image_path = os.path.join(image_dir, image_files[0])
    first_frame = cv2.imread(first_image_path)
    height, width, layers = first_frame.shape
    
    # Initialize the video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Specify the codec
    video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        frame = cv2.imread(image_path)
        if frame is None:
            print(f"Skipping {image_path}, cannot read image.")
            continue
        video_writer.write(frame)
    
    video_writer.release()
    print(f"Video saved to: {output_path}")

In [9]:

def interpolate_frames(image_1, image_2, times_to_interpolate, interpolator):
  input_frames = [str(image_1), str(image_2)]

  frames = list(
      util.interpolate_recursively_from_files(
          input_frames, times_to_interpolate, interpolator))
  return frames

## Main

In [10]:
# Preprocess images
files = natsorted([f for f in os.listdir(input_dir) if f.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))])
new_size = get_new_size(os.path.join(input_dir, files[0]))
create_empty_dir(output_dir)
create_empty_dir(processed_input_dir)
prepare_images(input_dir, processed_input_dir, new_size)
files = natsorted([f for f in os.listdir(processed_input_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])
len(files)


Preparing images:   0%|          | 0/24 [00:00<?, ?it/s]

Preparing images: 100%|██████████| 24/24 [00:00<00:00, 48.98it/s]


24

In [11]:
for file in files:
  timestamp = string_to_timestamp(file.split('.')[0])
  print(f"Filename: {file}, Datetime: {timestamp}")

Filename: 20160512_234123.jpg, Datetime: 2016-05-12 23:41:23
Filename: 20161214_220904.jpg, Datetime: 2016-12-14 22:09:04
Filename: 20171005_180654.jpg, Datetime: 2017-10-05 18:06:54
Filename: 20180814_184209.jpg, Datetime: 2018-08-14 18:42:09
Filename: 20190202_191947.jpg, Datetime: 2019-02-02 19:19:47
Filename: 20191122_084144.jpg, Datetime: 2019-11-22 08:41:44
Filename: 20191201_134035.jpg, Datetime: 2019-12-01 13:40:35
Filename: 20191226_214649.jpg, Datetime: 2019-12-26 21:46:49
Filename: 20200521_091805.jpg, Datetime: 2020-05-21 09:18:05
Filename: 20200803_091357.jpg, Datetime: 2020-08-03 09:13:57
Filename: 20200819_092657.jpg, Datetime: 2020-08-19 09:26:57
Filename: 20211230_090242.jpg, Datetime: 2021-12-30 09:02:42
Filename: 20220105_095857.jpg, Datetime: 2022-01-05 09:58:57
Filename: 20220130_204110.jpg, Datetime: 2022-01-30 20:41:10
Filename: 20220203_231940.jpg, Datetime: 2022-02-03 23:19:40
Filename: 20220531_124619.jpg, Datetime: 2022-05-31 12:46:19
Filename: 20220713_11305

In [12]:
# Iterate through pairs of consecutive frames

interpolator_model = interpolator.Interpolator(MODEL_PATH, None)
print('Total frames:', len(files))
for i in range(len(files) - 1):
  print('processing frame', i)
  start_filename = files[i]
  end_filename = files[i + 1]

  start_time = string_to_timestamp(start_filename.split('.')[0])
  end_time = string_to_timestamp(end_filename.split('.')[0])

  # Calculate the time difference in hours
  time_diff = (end_time - start_time).total_seconds() / frame_interval

  # Calculate times_to_interpolate for the required number of intermediate frames
  num_frames_needed = round(time_diff) - 1
  num_frames_needed = min(num_frames_needed, MAX_INTERPOLATED_FRAMES)
  times_to_interpolate = math.ceil(math.log2(num_frames_needed+1))  # n_intermediate = 2^k - 1

  frame_1 = os.path.join(processed_input_dir, start_filename)
  frame_2 = os.path.join(processed_input_dir, end_filename)
  output_frames = interpolate_frames(frame_1, frame_2, times_to_interpolate, interpolator_model)

  create_empty_dir(temp_interpolated_frames_dir)
  save_frames(output_frames, temp_interpolated_frames_dir)
  interpolated_files = natsorted([f for f in os.listdir(temp_interpolated_frames_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

  timestamps = np.linspace(start_time.timestamp(), end_time.timestamp(), len(interpolated_files))

  len(timestamps), num_frames_needed, time_diff
  # find required frames from generated frames (can be extra)
  chosen_timestamps = choose_evenly_spaced_timestamps(timestamps, num_frames_needed+2)
  for filename, timestamp in zip(interpolated_files, timestamps):
    if timestamp not in chosen_timestamps:
      continue
    new_filename = f"{timestamp_to_string(timestamp)}.{os.path.splitext(filename)[1]}"
    shutil.copyfile(os.path.join(temp_interpolated_frames_dir, filename), os.path.join(output_dir, new_filename))




2025-01-09 12:03:55.140540: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-01-09 12:03:55.160544: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-01-09 12:03:55.160596: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-01-09 12:03:55.164131: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-01-09 12:03:55.164187: I tensorflow/compile

Total frames: 24
processing frame 0


  0%|                                                                        | 0/31 [00:00<?, ?it/s]2025-01-09 12:03:57.168282: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [1,1080,711,3]
	 [[{{node inputs}}]]
2025-01-09 12:03:58.305775: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2025-01-09 12:04:02.410476: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
100%|███████████████████████████████████████████████████████████████| 31/31 [00:38<00:00,  1.25s/it]


processing frame 1


100%|███████████████████████████████████████████████████████████████| 31/31 [00:22<00:00,  1.36it/s]


...

100%|███████████████████████████████████████████████████████████████| 31/31 [00:22<00:00,  1.36it/s]


processing frame 21


100%|███████████████████████████████████████████████████████████████| 31/31 [00:22<00:00,  1.36it/s]


processing frame 22


100%|███████████████████████████████████████████████████████████████| 31/31 [00:22<00:00,  1.36it/s]


In [ ]:

# generate video from output directory
output_path = str(output_dir) + '.mp4'
fps = 24

write_video_from_images(output_dir, output_path, fps)